# 0 - Before starting

TCP SDK allows you to :
- manage safe connections to the API
- easily send http requests to the API thanks to the method query()

## a. Requirements

In [ ]:
import requests
import json
import os
from requests.auth import HTTPBasicAuth
from tcp import client
import tcp

## b. Host Definition

In [ ]:
host="https://unstable.thecrossproduct.xyz/v1"

## c. Connection information

In [ ]:
email=""
password = ""

# 1 - Authentification

## a. User Login

User login is done by client object instanciation:

In [ ]:
api=client(host=host,usermail=email,passwd=password)

You can now retrieve your token as follows:

In [ ]:
token=api.token
print(token)

TIPS: Once you retrieved your token, you can add the token to the env variable "TCP_API_TOKEN". 
After that, you won't need to provide email and password to create a connection

### TIPS - Finding endpoints - help method

TCP sdk provides a help method providing a list of all endpoints available and specific functions.

It can be called:

#### - on the whole module:

In [ ]:
api.help()

#### - on a section

In [ ]:
api.query().app.help()

#### - on any endpoint

In [ ]:
api.query().app.run.help()

____________________________________________________________________________________________________
Now, let's go back to the "auth" section:

## b. Get user's informations

This endpoints provide user informations such as id, mail, creation date, associated groups...

In [ ]:
#Method=GET
#Endpoints=auth/get/

user_info=api.query().auth.get()
print(user_info)

## c. Get token's informations

This endpoint verifies the validity of your token and provides some information about it, including token id, user id, expiration date, eventual activation limits, number of uses, scope authorized ...

In [ ]:
#Method=GET
#Endpoints=auth/validate_auth_token/

print(api.query().auth.validate_auth_token.get())


## d. User logout

In [ ]:
#Method=GET
#Endpoints=auth/logout

api.query().auth.logout.get()

# 2 - Manage your licences

## a. Get licence information

This endpoints provides a list of licenses associated with your account.

In [ ]:
#Method=GET
#Endpoints=lics/

print(api.query().lics.get())

# 3 - Manage your data

This section, allows you to check your data, upload new data from your local repository to the S3 storage or, on the contrary, download data from S3 storage to your local repository.

In order to make it user friendly, we created the functions upload() and download().  

## a. Upload Data

In [ ]:
# Arguments definition
s3_target_uri="test_upload"
local_file_path="/home/demo/DonneesTests/demo/01_Sources/voie1.dxf"
file_size=str(os.path.getsize(local_file_path))

In [ ]:
#returns True if successfull
print(api.upload(local_file_path,s3_target_uri))

## b. Download data

In [ ]:
# Arguments definition
target_local_path="/home/demo/DonneesTests/demo/test_dl/voie1.dxf"

In [ ]:
#returns True if successfull
print(api.download(s3_target_uri,target_local_path))

## c. List your TCP data

In [ ]:
#Method=GET
#Endpoints=data/

print(api.query().data.get())

# 4 - TCP solutions

## a. List TCP Solutions

This endpoints provides a list of products that are available to your account, sorted by domains.

In [ ]:
#Method=GET
#Endpoints=app/

print(api.query().app.get())

## b. List Available remotes

The following endpoint provides a comprehensive list of all available Scaleway computing resources, including information such as the number of cores, size of random access memory (RAM), and hourly cost. In addition, if you have declared remotes, they will also appear in this list.

In [ ]:
#Method=GET
#Endpoints=app/remote/scw
print(api.query().app.remote.scw.get())


## c. Specific informations for Domain

This endpoint provides a detailed documentation on how to use the solution for a given application. It lists all the available endpoints and their corresponding functionalities. Additionally, it provides detailed information on the required JSON body for each endpoint. The fields in the JSON body are described with their name, type, and a brief description of their purpose. This documentation serves as a comprehensive guide for you to effectively use the solution.

In [ ]:
#Method=GET
#Endpoints=app/info
#Url argument= Domain="railway"& App="classification"

print(api.query().app.info.get(Domain="railway",App="classification").decode('utf8'))

## d. Run TCP application

This endpoint is used to initiate a process, where a solution is executed with a given set of parameters.

At least two parameters are required: "inputs" and "output-prefix". Those parameters must strictly adhere to the documentation provided by the endpoint GET /v1/app/info.

Additionally, you can use the parameter "pool" to select a specific architecture on which to run the process. This parameter is a list of instances, that you have selected. The most suitable will be selected in this list. Refer to the endpoint GET /v1/app/remote. If the parameter "pool" is omitted, the run will be run on a small architecture by default.

Upon successful initiation, an ID is returned to you. This ID will be used in subsequent requests to check the status of the process and retrieve the results.

In [ ]:
#inputs definition
clouds=["demo/cloud1.laz"]
trj=["demo/trajectory.dxf"]
output="demo/classif"

In [ ]:
#Method=POST
#Endpoints=app/info
#Url argument= Domain="railway"& App="classification"
#data body= {"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]}

resp=api.query().app.run.post({"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]},Domain="railway",App="classification")
if resp.status_code==200:
    print(resp.text)
else:
    print(resp)

## e. Check process status & download results

Use this endpoint to check the status of a process and retrieve its outputs once it has finished. The following fields are returned: process id, user id, application used, domain, dates of launch and termination, state of the process.

The "outputs" field in the response is a dictionary that contains the files generated by the process. The keys in the dictionary represent the names of the files, and the corresponding values are URLs for download.

The "outputs" field in the response is a dictionary that contains the files generated by the process. The keys in the dictionary represent the names of the files, and the corresponding values are URLs for download.

If the process was initiated with the POST v1/app/token endpoint, the "QUOTE" subfield will indicate the number of tokens that are to be consumed

In [ ]:
#Method=GET
#Endpoints=app/status
#id_process = resp.json()['id']
id_process = resp['id']

print(api.query().app.status.get(Process=id_process))


## f. List Processes

Use this endpoint to retrieve information about all the processes you currently have, including their status and output.

In [ ]:
#Method=GET
#Endpoints=app/list/Process

print(api.query().app.list.Process.get())